In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
df=pd.read_csv('train.csv')

In [35]:
df.shape

(404290, 6)

In [36]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [39]:
new_df=df.sample(30000)

In [44]:
# Count of null values
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [45]:
# Count of empty rows
new_df.duplicated().sum()

0

In [46]:
ques_df=new_df[['question1','question2']]
ques_df.head()

,question1,question2
44503,What methods did you use to skip school?,20s (age): What tips or methods have you used ...
106114,Why are there boroughs and not just separate c...,Can someone list all the rappers from each bor...
54259,During the formation of Aqueous solution of So...,During the growth of microbes on nitrogen free...
61352,How does NBA system work?,How does overtime work in the NBA?
81621,What will be the impact of Brexit on the US ec...,What impact will Brexit have on the US economy?


In [47]:
# Merge Texts
from sklearn.feature_extraction.text import CountVectorizer
questions=list(ques_df['question1'])+list(ques_df['question2'])
cv=CountVectorizer(max_features=3000) #Most used 3000 words feature
q1_arr,q2_arr=np.vsplit(cv.fit_transform(questions).toarray(),2) #We get BOW for every questions

In [48]:
# Creating a new dataframe temp_df consisting 
temp_df1=pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2=pd.DataFrame(q2_arr, index=ques_df.index)
temp_df=pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(30000, 6000)

In [50]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
44503,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0
106114,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
61352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223590,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
251290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
219872,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:
temp_df['is_duplicate']=new_df['is_duplicate']

In [55]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)*100

73.81666666666666

In [65]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [66]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7258333333333333